In [2]:
import sys; sys.path.append("../")

import pandas as pd

from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)
from lib.date import combine_date_columns
from clean.new_orleans_harbor_pd_pprr import clean_personnel_2008, clean_personnel_2020
from match.new_orleans_harbor_pd import prepare_post_data, match_uid_with_cprr
from lib.columns import rearrange_personnel_columns

In [3]:
df20 = clean_personnel_2020()
df20

,first_name,last_name,rank_desc,hourly_salary,middle_initial,effective_date,agency,data_production_year,uid,perhist_uid,...,resign_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,rank_year,rank_month,rank_day
0,kendall,turner,police officer 1-a,10.10,r,1/3/2007,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,c3dc1f7275ff3681ca673813d786b798,...,,2007,1,3,2007,1,3,2007,1,3
1,kendall,turner,police officer 2-a,15.18,r,1/4/2008,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,13bf83f9ed69849fc2ad61e5329adca5,...,,2007,1,3,2008,1,4,2008,1,4
2,kendall,turner,police officer 2-a,17.08,r,10/1/2012,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,043c9de3e938d8cd2525def644ad3c64,...,,2007,1,3,2012,10,1,,,
3,kendall,turner,police officer 2-a,17.76,r,10/1/2013,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,59fccb5c71605c1916f8939aa6aa1381,...,,2007,1,3,2013,10,1,,,
4,kendall,turner,police officer 2-a,18.47,r,10/1/2014,New Orleans Harbor PD,2020,01cf5194afc556e5b49526c3636e2362,d3e867074632239a401ae0f9b7bcb0b5,...,,2007,1,3,2014,10,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,linda,st cyr,police officer 3-a,19.50,l,10/1/2007,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,2c1c0b941a8c74083a15df59e0c1cbc7,...,31,1982,12,27,2007,10,1,,,
898,linda,st cyr,police officer 3-a,22.31,l,10/1/2008,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,8162e05dc991530289bc4115e449d25a,...,31,1982,12,27,2008,10,1,,,
899,linda,st cyr,police officer 3-a,21.45,l,11/1/2007,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,f31cde8d053c5701bc9db56facc2fa21,...,31,1982,12,27,2007,11,1,,,
900,linda,st cyr,police officer 3-a,18.75,l,7/1/2007,New Orleans Harbor PD,2020,f9201bbb609a5755066a4bc23b7bb7e6,eea3e1b2bf424806c2a2ec9c4c363fb1,...,31,1982,12,27,2007,7,1,,,


In [4]:
df08 = clean_personnel_2008()
df08

,first_name,middle_initial,last_name,rank_desc,effective_date,hourly_salary,agency,data_production_year,uid,hire_year,...,hire_day,resign_year,resign_month,resign_day,pay_effective_year,pay_effective_month,pay_effective_day,rank_year,rank_month,rank_day
0,cheryl,e,st pe,police corporal--port,2/28/1992,9.99,New Orleans Harbor PD,2008,000526fa05739e97b61343513a92dbc7,1981,...,19,1992,2,28,1992,2,28,1992,2,28
1,larry,m,francingues,police officer 2--port,1/19/2002,11.32,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,1999,...,19,2005,12,19,2002,1,19,2002,1,19
2,larry,m,francingues,police officer 2--port,1/19/2003,11.77,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,1999,...,19,2005,12,19,2003,1,19,,,
3,larry,m,francingues,police officer 2--port,1/19/2004,12.24,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,1999,...,19,2005,12,19,2004,1,19,,,
4,larry,m,francingues,police officer 2-a,1/19/2005,12.73,New Orleans Harbor PD,2008,010c27441d135085258ae415c22843ae,1999,...,19,2005,12,19,2005,1,19,2005,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,antoine,a,derouen jr,police officer 2--port,3/8/2003,12.75,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,1998,...,8,2006,12,11,2003,3,8,,,
163,antoine,a,derouen jr,police officer 2--port,3/8/2004,13.26,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,1998,...,8,2006,12,11,2004,3,8,,,
164,antoine,a,derouen jr,police officer 2--port,3/8/2005,13.79,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,1998,...,8,2006,12,11,2005,3,8,,,
165,antoine,a,derouen jr,police officer 2-a,3/8/2006,14.34,New Orleans Harbor PD,2008,fb21c5c11c57efd44890f8e2c4fbe7f7,1998,...,8,2006,12,11,2006,3,8,2006,3,8


In [9]:
df20.loc[df20.uid == '2b3675fa11b85f275f110cc30834d5fb']

,first_name,last_name,rank_desc,hourly_salary,middle_initial,effective_date,agency,data_production_year,uid,perhist_uid,...,resign_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,rank_year,rank_month,rank_day
179,michael,st cyr,police sergeant-a,28.34,p,1/1/2018,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,9cd2d411400a4fc105cea97f9838e33b,...,20,1996,12,9,2018,1,1,2018,1,1
180,michael,st cyr,police sergeant-a,24.69,p,10/1/2012,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,f50b1c265367768b6674105d0a1f53ae,...,16,1996,12,9,2012,10,1,,,
181,michael,st cyr,police sergeant-a,25.68,p,10/1/2013,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,fc5681f3685802c178cf99eef0e1ae4b,...,17,1996,12,9,2013,10,1,,,
182,michael,st cyr,police sergeant-a,26.71,p,10/1/2014,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,931c346b23975ec48e5554145ea33b92,...,18,1996,12,9,2014,10,1,,,
183,michael,st cyr,police sergeant-a,27.78,p,10/1/2015,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,5bead6e1d87582ede7326e7d010f88fd,...,19,1996,12,9,2015,10,1,,,
184,michael,st cyr,police officer 3-a,19.11,p,11/1/2007,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,276813cba587dcbf3fcf24dccfabe9f8,...,11,1996,12,9,2007,11,1,2007,11,1
185,michael,st cyr,police corporal--port,13.15,p,12/22/2000,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,ec978a999cd108f1b8685420041e78e4,...,2,1996,12,9,2000,12,22,2000,12,22
186,michael,st cyr,police sergeant-a,23.74,p,6/30/2010,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,fe8c5cde5a75a6b7b09d183dd054fe6b,...,15,1996,12,9,2010,6,30,2010,6,30
187,michael,st cyr,police sergeant-a,22.83,p,6/7/2010,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,7415dd0ff2318e6142953c6320c950a8,...,14,1996,12,9,2010,6,7,,,
188,michael,st cyr,police corporal--port,13.68,p,6/9/2002,New Orleans Harbor PD,2020,2b3675fa11b85f275f110cc30834d5fb,d506ebc39d258bb5ffeba977aed6be79,...,3,1996,12,9,2002,6,9,2002,6,9
